1 Instanciation spark session

In [1]:
from pyspark import SparkContext
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import configparser

In [2]:
spark=SparkSession.builder\
                  .master("local")\
                  .appName("Bristol")\
                  .getOrCreate()

2 Creéation du fichier propertie.conf

In [3]:
import configparser
config = configparser.ConfigParser()
config.read('properties.conf')
path_to_input_data = config['Bristol-City-bike']['Input_data']
path_to_output_data = config['Bristol-City-bike']['Output_data']
num_partition_kmeans = int(config['Bristol-City-bike']['Kmeans_level'])

In [4]:
pip install configparser

Note: you may need to restart the kernel to use updated packages.


Question 3 Importation json avec spark

In [5]:
bristol = spark.read.json(path_to_input_data)

In [6]:
bristol.show(3)

+--------------------+----------+----------+--------------------+------+
|             address|  latitude| longitude|                name|number|
+--------------------+----------+----------+--------------------+------+
|Lower River Tce /...|-27.482279|153.028723|122 - LOWER RIVER...|   122|
|Main St / Darragh St| -27.47059|153.036046|91 - MAIN ST / DA...|    91|
|Sydney St Ferry T...|-27.474531|153.042728|88 - SYDNEY ST FE...|    88|
+--------------------+----------+----------+--------------------+------+
only showing top 3 rows



Question 4 Dataframe Kmeans

In [8]:
Kmeans_df =  bristol.select(col("latitude"),col("longitude"))

In [9]:
Kmeans_df.show(2)

+----------+----------+
|  latitude| longitude|
+----------+----------+
|-27.482279|153.028723|
| -27.47059|153.036046|
+----------+----------+
only showing top 2 rows



Question 5 Kmeans

In [10]:
# 5 Kmeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
features = ("longitude","latitude")
kmeans = KMeans().setK(num_partition_kmeans).setSeed(1)
assembler = VectorAssembler(inputCols=features,outputCol="features")
dataset=assembler.transform(Kmeans_df)
model = kmeans.fit(dataset)
fitted = model.transform(dataset)

Question 6 : Colonnes de fitted

In [25]:
fitted.columns

['latitude', 'longitude', 'features', 'prediction']

In [26]:
fitted

DataFrame[latitude: double, longitude: double, features: vector, prediction: int]

In [27]:
fitted.show()

+----------+----------+--------------------+----------+
|  latitude| longitude|            features|prediction|
+----------+----------+--------------------+----------+
|-27.482279|153.028723|[153.028723,-27.4...|         2|
| -27.47059|153.036046|[153.036046,-27.4...|         2|
|-27.474531|153.042728|[153.042728,-27.4...|         1|
|-27.461881|153.046986|[153.046986,-27.4...|         1|
|-27.469658|153.016696|[153.016696,-27.4...|         2|
| -27.48172| 153.00436|[153.00436,-27.48...|         0|
|-27.493626|153.001482|[153.001482,-27.4...|         0|
|-27.476076|153.002459|[153.002459,-27.4...|         0|
|-27.493963|153.011938|[153.011938,-27.4...|         0|
|-27.482197|153.020894|[153.020894,-27.4...|         2|
|-27.465226|153.050864|[153.050864,-27.4...|         1|
|-27.468447|153.024662|[153.024662,-27.4...|         2|
|-27.473021|153.025988|[153.025988,-27.4...|         2|
|-27.457825|153.036866|[153.036866,-27.4...|         1|
| -27.48148| 153.02368|[153.02368,-27.48...|    

Question 7 : Les moyennes par groupe
    LES RESULTATS SONT LES MEMES

In [34]:
#Methode DSL
fitted.groupBy(col("prediction")).mean("latitude", "longitude").show()

+----------+-------------------+------------------+
|prediction|      avg(latitude)|    avg(longitude)|
+----------+-------------------+------------------+
|         1|-27.460240636363633|153.04186302272726|
|         2| -27.47255990624999|   153.02594553125|
|         0|-27.481218536585374|153.00572882926832|
+----------+-------------------+------------------+



In [35]:
#Methode SQL
fitted.createOrReplaceTempView("fit")

In [40]:
spark.sql("""SELECT prediction, avg(latitude) as Alatitude, avg(longitude) as Alongitude FROM fit GROUP BY prediction """).show()

+----------+-------------------+------------------+
|prediction|          Alatitude|        Alongitude|
+----------+-------------------+------------------+
|         1|-27.460240636363633|153.04186302272726|
|         2| -27.47255990624999|   153.02594553125|
|         0|-27.481218536585374|153.00572882926832|
+----------+-------------------+------------------+



Question 9 : Exportation FITTED

In [44]:
fitted.drop(col("features")).toPandas().to_csv(path_to_output_data+"fitt.csv")